In [3]:
!pip install pytorch-pretrained-bert pytorch-nlp


In [4]:
cd /content/drive/MyDrive/CSCI544/PROJECT

/content/drive/MyDrive/CSCI544/PROJECT


In [19]:
# BERT imports
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
import json


In [6]:
# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)


'Tesla P100-PCIE-16GB'

In [53]:
#load cleaned data 
data = pd.read_csv("cleaned_data.csv", sep=",")


In [54]:
data["category"] = data["category"].apply(lambda x: x[1:-1].split(",")[0][1:-1])


In [57]:
# convert genders into label 
# 1 = man, 0= woman
data["gender"] = data["gender"].apply(lambda x: 1 if x == "man" else 0)

In [58]:
data.head()

,text,gender,category
0,till with that wealth he pays his soldiery.,1,work
1,"better to die than leave his banneret.""",1,achievement
2,"yet which might win they knew not, in his thou...",1,achievement
3,"then find him men, his company are worth.""",1,work
4,"at the sword's point he yet shall pay our meed.""",1,work


In [59]:

# count the number of genders and category in the data set 
data["gender"].value_counts()

1    35987
0     9989
Name: gender, dtype: int64

In [56]:
data["category"].value_counts()

achievement    21669
work           17018
home            7289
Name: category, dtype: int64

In [61]:
# note there is a gender imbalance 
# we create the testing set by sampling 2000 instances from both genders 
data_male = data[data["gender"] == 1]
data_female = data[data["gender"] == 0]
test_male = data_male.sample(2000)
test_female = data_female.sample(2000)


In [72]:
test = pd.concat([test_male, test_female])
test = test.sample(frac = 1).reset_index(drop = True)

In [65]:
# drop the sampled instances from data 
data = data.drop(test_male.index)
data = data.drop(test_female.index)

In [67]:
# train_test split 
train, validation = train_test_split(data, train_size=0.7)

In [75]:
train = train.reset_index(drop = True)
validation = validation.reset_index(drop = True)

In [76]:
train

,text,gender,category
0,"<p>but reflection brought better feelings, and...",0,achievement
1,"he worked hard and earnestly, and dealt his bl...",1,work
2,"during his working career, 100l. per annum; an...",1,work
3,"with slow and staggering effort he arose,",1,achievement
4,than i could work miracles: he writes a good ...,1,work
...,...,...,...
29378,"good for a fortnight, he sent upstairs for gri...",1,home
29379,"""i feel better to-night,"" he murmured, abrup...",1,achievement
29380,"and many a pink-white baby girl, the queen of ...",0,home
29381,the helping hand he did so with a delicacy and...,1,achievement


In [77]:
validation

,text,gender,category
0,outside; and sate down to as good a supper amo...,1,home
1,was preparing an edition of all his latin work...,1,home
2,lingered till she saw the last of their compan...,0,work
3,about doing so. making a great effort she did ...,0,achievement
4,"had worked in red, blue, and brown wools when ...",0,work
...,...,...,...
12588,road-mender tried to get a peep at secret weap...,1,achievement
12589,had tried to put into effect with the aid of h...,1,achievement
12590,"black bill, because he always works alone and ...",1,work
12591,"the little old man resumed, ""that even now i c...",1,work


In [78]:
test

,text,gender,category
0,"industry, as well as of his aggressive work of...",1,work
1,"paid no heed; and he had tied him up, and he p...",1,work
2,she tried to smile for my sake. it was settle...,0,achievement
3,he had only two hours' work to do to open comm...,1,work
4,people in whose company she found herself had ...,0,work
...,...,...,...
3995,"i do not. he is quite correct, and infinitely...",1,achievement
3996,who was the other? angele was carried to her...,0,home
3997,rachel immediately tried to qualify what she h...,0,achievement
3998,mr. clemens had tried to be funny but had fail...,1,achievement


In [79]:
# save the datasets to file 
train.to_csv("train.csv")
validation.to_csv("validation.csv")
test.to_csv("test.csv")